In [1]:
import web3 
import math
from sklearn.datasets import fetch_california_housing
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
import warnings
from UniswapV3TradingFramework import priceToSqrt,V3TraderFramework
from position import Position
from swap import Swap 
warnings.filterwarnings('ignore', category=UserWarning)

# WBTC/USDC Pool 

In [6]:
pool_address = "0x99ac8cA7087fA4A2A1FB6357269965A2014ABc35"
fee_tier = 0.003
graph_url = "https://gateway.thegraph.com/api/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"
v3_trader = V3TraderFramework(url=graph_url, pool_address = pool_address, poolFeeTier=fee_tier)


In [7]:
swaps = v3_trader.retrieveSwapsData(from_time=1749877200, to_time=1749963600)

IndexError: list index out of range